In [1]:
import pandas as pd
import random


### Prime Number Functions

**`is_prime(n)`**  
- Checks if a number `n` is prime.
- Returns `True` if `n > 1` and divisible only by 1 and itself, else `False`.

**`next_prime(n)`**  
- Finds the next prime number ≥ `n`.
- Uses `is_prime` to incrementally check numbers until a prime is found.


In [2]:
def is_prime(n):
    if n <= 1:
        return False
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
    return True

def next_prime(n):
    while not is_prime(n):
        n += 1
    return n


### Loading and Merging Datasets

1. **Load Datasets**  
   - `ratings`: Contains user ratings for movies (`rating.csv`).  
   - `movies`: Contains movie information (`movie.csv`).

2. **Merge Datasets**  
   - Combine `ratings` and `movies` on the `movieId` column using an inner join.  
   - Result: A unified DataFrame `df` with movie ratings and details.

3. **Preview the Data**  
   - Display the first few rows of the merged DataFrame using `df.head()`.


In [ ]:
ratings = pd.read_csv('rating.csv')  
movies = pd.read_csv('movie.csv')    

df = pd.merge(ratings, movies, on='movieId', how='inner')


In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df.info()


In [ ]:
df.nunique()

### MinHash Implementation from Scratch

**`minhash_scratch(user_movie_dict, perm)`**

1. **Purpose**  
   - Generate MinHash signatures for each user based on the movies they have rated.  

2. **Parameters**  
   - `user_movie_dict`: Dictionary where keys are users and values are lists of movie IDs.  
   - `perm`: Number of hash functions (permutations) to use for the MinHash computation.

3. **Logic**  
   - **Generate Unique Movie Set:** Extract all unique movie IDs across all users.  
   - **Hash Functions:**  
     - Generate `perm` random coefficients `a` and `b` for hash functions.  
     - Use a prime number `p` greater than the total number of movies to ensure good hashing.  
   - **Initialization:** Create a dictionary to store MinHash signatures (`user_sign`), initialized with infinity (`float('inf')`) for each permutation.  
   - **Hash Computation:**  
     - For each user and their rated movies:  
       - Compute the hash value for each movie using \( \text{hash} = (a \cdot \text{movieId} + b) \% p \).  
       - Update the user's MinHash signature with the smallest hash value for each permutation.

4. **Return**  
   - `user_sign`: Dictionary containing MinHash signatures for each user.



In [8]:
def minhash_scratch(user_movie_dict, perm):
    all_movies = set(movie for movies in user_movie_dict.values() for movie in movies)

    a_vals = [random.randint(1,len(all_movies)) for i in range(perm)]
    b_vals = [random.randint(0,len(all_movies)) for i in range(perm)]

    p = next_prime(len(all_movies) + 1)

    user_sign = {user : [float('inf')] * perm for user in user_movie_dict}

    for user, movies in user_movie_dict.items():
        for movie in movies:
            for i in range(perm):
                hash_value = (a_vals[i] * movie + b_vals[i]) % p ## Why we need the smallest hash values??
                if hash_value < user_sign[user][i]:
                    user_sign[user][i] = hash_value
    
    return user_sign

### Creating `user_movie_dict`

- **Group by `userId`:** Groups the DataFrame by users.
- **Extract `movieId`:** Collects all movie IDs each user has rated.
- **Convert to List:** Combines the movie IDs into lists for each user.
- **Convert to Dictionary:** Creates a dictionary where:
  - Keys: `userId` (user identifiers).
  - Values: Lists of `movieId` (movies rated by the user).

**Result:**  
A dictionary `user_movie_dict` where each user is associated with the list of movies they have rated.


In [9]:
user_movie_dict = df.groupby('userId')['movieId'].apply(list).to_dict()

In [ ]:
user_movie_dict

### Locality-Sensitive Hashing (LSH) from Scratch

**`lsh_scratch(minhash_signatures, num_bands, rows_per_band)`**

1. **Purpose**  
   - Implements the LSH algorithm to group users with similar MinHash signatures into the same buckets and identify candidate pairs for similarity checks.

2. **Parameters**  
   - `minhash_signatures`: Dictionary of users and their MinHash signatures.  
   - `num_bands`: Number of bands to divide the signatures into.  
   - `rows_per_band`: Number of rows in each band.

3. **Logic**  
   - **Assertion Check:** Ensures the total number of rows in bands matches the signature length.  
   - **Initialization:**
     - `buckets`: Stores all users grouped by their bucket hash.
     - `candidate_pairs`: Holds all user pairs identified as candidates for similarity.  
   - **Band Processing:**  
     - For each band, slice the signature into sub-sections (`start_idx:end_idx`).  
     - Hash each band to a `band_hash` and group users with the same hash into `band_buckets`.
     - Print debug information: Band number, hash, and grouped users.
   - **Update Buckets:**  
     - Add the users in `band_buckets` to the global `buckets` structure.  
   - **Identify Candidate Pairs:**  
     - For users in the same `band_buckets` group, generate all possible user pairs.

4. **Return Values**  
   - `buckets`: Dictionary containing users grouped by bucket hash.  
   - `candidate_pairs`: Set of user pairs identified as candidates for further similarity evaluation.

5. **Key Steps:**
   - **Band Hashing:**  
     Users with similar MinHash signatures fall into the same bucket, increasing the likelihood of finding similar users.
   - **Candidate Pair Generation:**  
     User pairs in the same bucket are flagged as potentially similar for detailed similarity checks.

**Debugging Output:**  
For each band, the band index, hash values, and associated users are printed for tracking progress.


In [13]:
from collections import defaultdict

def lsh_scratch(minhash_signatures, num_bands, rows_per_band):
    assert num_bands * rows_per_band == len(list(minhash_signatures.values())[0]), \
        "ERROR: num_bands * rows_per_band does not match the signature length."

    buckets = defaultdict(list)
    candidate_pairs = set()

    for band_idx in range(num_bands):
        band_buckets = defaultdict(list)

        for user, signature in minhash_signatures.items():
            start_idx = band_idx * rows_per_band
            end_idx = start_idx + rows_per_band
            band = tuple(signature[start_idx:end_idx])

            band_hash = hash(band)  # Built-in Python hash
            band_buckets[band_hash].append(user)

            print(f"Band {band_idx}, Hash {band_hash}: {band_buckets[band_hash]}")

        # Add users to general buckets
        for band_hash, users in band_buckets.items():
            buckets[band_hash].extend(users)

        # Check for candidate pairs
        for bucket_users in band_buckets.values():
            if len(bucket_users) > 1:
                for i in range(len(bucket_users)):
                    for j in range(i + 1, len(bucket_users)):
                        candidate_pairs.add((bucket_users[i], bucket_users[j]))

    return buckets, candidate_pairs


In [11]:
minhash_signatures = minhash_scratch(user_movie_dict, 32)

In [ ]:
minhash_signatures

In [ ]:
num_bands = 8
rows_per_band = 4

buckets, candidate_pairs = lsh_scratch(minhash_signatures, num_bands, rows_per_band)



In [ ]:
buckets

In [ ]:
candidate_pairs

In [ ]:
len(buckets)

In [ ]:
len(candidate_pairs)

### Finding Similar Users

**Function:** `find_similar_users(candidate_pairs, user_movie_dict)`

This function identifies and ranks similar users based on the number of movies they have in common, leveraging the candidate pairs generated by the LSH algorithm.

---

#### **1. Parameters**
- **`candidate_pairs`**: Set of user pairs identified as potentially similar.  
- **`user_movie_dict`**: Dictionary mapping users to the list of movies they have rated.

---

#### **2. Logic**
- **Step 1: Calculate Common Movies**  
  - For each user pair in `candidate_pairs`:  
    - Retrieve their respective movie lists and convert them into sets.  
    - Compute the intersection of the sets to find common movies.  
    - If the intersection is non-empty, add the pair to the `similar_users` dictionary, along with the count of common movies.  

- **Step 2: Sort Similar Users**  
  - For each user, sort their list of similar users in descending order of the number of common movies.

---

#### **3. Return Value**
- **`similar_users`**:  
  A dictionary where each user is mapped to a list of tuples.  
  - Each tuple contains:  
    1. A similar user.  
    2. The number of movies they have in common.  
  - The list is sorted by the number of common movies (highest first).

---

#### **4. Example Workflow**

**Input:**  
```python
candidate_pairs = {('User1', 'User2'), ('User1', 'User3')}
user_movie_dict = {
    'User1': [101, 102, 103],
    'User2': [101, 103, 104],
    'User3': [102, 105]
}



---

**Output:**  
```python
{
    'User1': [('User2', 2), ('User3', 1)],
    'User2': [('User1', 2)],
    'User3': [('User1', 1)]
}


In [23]:
def find_similar_users(candidate_pairs, user_movie_dict):
    similar_users = defaultdict(list)  

    for user1, user2 in candidate_pairs:
        movies_user1 = set(user_movie_dict[user1])
        movies_user2 = set(user_movie_dict[user2])
        
        common_movies = movies_user1.intersection(movies_user2)
        
        if len(common_movies) > 0:
            similar_users[user1].append((user2, len(common_movies)))
            similar_users[user2].append((user1, len(common_movies)))

    for user in similar_users:
        similar_users[user] = sorted(similar_users[user], key=lambda x: x[1], reverse=True)
    
    return similar_users


### Movie Recommendation Function

**Function:** `recommend_movies(user, similar_users, user_movie_dict, rating_dict, max_recommendations=5)`

This function recommends movies to a user based on the preferences of similar users.

---

#### **1. Parameters**
- **`user`**: The target user for whom the recommendations are generated.  
- **`similar_users`**: A dictionary mapping users to their similar users and the number of common movies.  
- **`user_movie_dict`**: A dictionary mapping users to the list of movies they have rated.  
- **`rating_dict`**: A dictionary mapping users to their movie ratings as `{movieId: rating}`.  
- **`max_recommendations`**: The maximum number of movies to recommend (default is 5).

---

#### **2. Logic**
1. **Identify Movies Already Watched**  
   - Extract the list of movies the target user has already watched to avoid recommending these.

2. **Iterate Over Similar Users**  
   - For each similar user:
     - Identify the movies they have watched but the target user has not (`unseen_movies`).

3. **Aggregate Ratings for Unseen Movies**  
   - For each unseen movie:
     - If it is not already in the recommendation list, add it with the similar user's rating.
     - If it is already in the list, add the rating to its existing score.  
   - This ensures that movies recommended by multiple similar users receive a higher score.

4. **Sort Recommendations**  
   - Sort the unseen movies by their aggregated scores in descending order.

5. **Return Top Recommendations**  
   - Return the top `max_recommendations` movies from the sorted list.

---

#### **3. Return Value**
- A list of movie IDs representing the top recommended movies, sorted by their scores.

---

#### **4. Example Workflow**

**Input:**  
```python
user = 'User1'
similar_users = {
    'User1': [('User2', 5), ('User3', 3)]
}
user_movie_dict = {
    'User1': [101, 102],
    'User2': [102, 103],
    'User3': [103, 104]
}
rating_dict = {
    'User2': {102: 4, 103: 5},
    'User3': {103: 3, 104: 4}
}


---

**Output:**  
```python
[103, 104]


In [20]:
def recommend_movies(user, similar_users, user_movie_dict, rating_dict, max_recommendations=5):
    user_movies = set(user_movie_dict[user]) 
    recommendations = {}

    if user not in similar_users:
        return []  
    
    for similar_user, _ in similar_users[user]:
        similar_user_movies = set(user_movie_dict[similar_user])
        
        unseen_movies = similar_user_movies - user_movies
        
        for movie in unseen_movies:
            if movie not in recommendations:
                recommendations[movie] = rating_dict[similar_user].get(movie, 0)  
            else:
                recommendations[movie] += rating_dict[similar_user].get(movie, 0) 
    
    sorted_recommendations = sorted(recommendations.items(), key=lambda x: x[1], reverse=True)
    
    return [movie for movie, _ in sorted_recommendations[:max_recommendations]]


### Creating `rating_dict`

**Purpose:**  
Creates a dictionary mapping each `userId` to their rated `movieId` and corresponding `rating`.

**Steps:**  
1. **Group by `userId`:** Groups movies and ratings for each user.  
2. **`zip` and `dict`:** Combines `movieId` and `rating` into a dictionary for each user.  
3. **Convert to `dict`:** Creates a nested dictionary.

**Result:**  
A dictionary in the format:
```python
{
    'User1': {101: 4.0, 102: 5.0},
    'User2': {103: 3.5, 104: 4.0},
    ...
}


In [21]:
rating_dict = (
    df.groupby('userId')
    .apply(lambda x: dict(zip(x['movieId'], x['rating'])))
    .to_dict()
)


In [24]:
similar_users = find_similar_users(candidate_pairs, user_movie_dict)

In [ ]:
similar_users

In [ ]:
user_id = 2 
recommended_movies = recommend_movies(user_id, similar_users, user_movie_dict, rating_dict)

print(f"Recommended Movies for User {user_id}: {recommended_movies}")

In [ ]:
user_id = 1 
recommended_movies = recommend_movies(user_id, similar_users, user_movie_dict, rating_dict)

print(f"Recommended Movies for User {user_id}: {recommended_movies}")